In [1]:
import os
import vtk

set the dicom directory

In [2]:
fileDir = './CBCT_files/'

see list of files in directory

In [3]:
print('number of dcm files : ' + str(len(os.listdir(fileDir))))

number of dcm files : 349


In [4]:
reader = vtk.vtkDICOMImageReader();
reader.SetDirectoryName(fileDir)
reader.Update()

In [5]:
threshold = vtk.vtkImageThreshold()
threshold.SetInputConnection(reader.GetOutputPort())
#threshold.ThresholdByLower(50)  # remove all soft tissue # Not used here
threshold.ThresholdBetween(400,1000)
threshold.ReplaceInOn()
threshold.SetInValue(0)  # set all values below 400 to 0
threshold.ReplaceOutOn()
threshold.SetOutValue(1)  # set all values above 400 to 1
threshold.Update()

In [6]:
boneExtractor = vtk.vtkMarchingCubes();
boneExtractor.ComputeScalarsOff();
boneExtractor.SetInputConnection(threshold.GetOutputPort());
#skinExtractor.SetValue(1, 80);
boneExtractor.GenerateValues(1, 1, 1)
boneExtractor.Update();

In [7]:
smoother = vtk.vtkSmoothPolyDataFilter();
smoother.SetInputConnection(boneExtractor.GetOutputPort());
smoother.SetNumberOfIterations(10);
smoother.SetRelaxationFactor(0.1);
smoother.SetFeatureAngle(1);
smoother.FeatureEdgeSmoothingOff();
smoother.BoundarySmoothingOn();
smoother.SetConvergence(0);
smoother.Update();

In [8]:
decimate = vtk.vtkDecimatePro();
decimate.SplittingOff();
decimate.SetErrorIsAbsolute(5);
decimate.SetFeatureAngle(1);
decimate.PreserveTopologyOn();
decimate.BoundaryVertexDeletionOff();
decimate.SetDegree(1); 
decimate.SetInputData(smoother.GetOutput()); 
decimate.SetTargetReduction(0.10);
decimate.SetMaximumError(0.001);
decimate.Update();

In [9]:
normalsGenerator = vtk.vtkPolyDataNormals();
normalsGenerator.SetInputData(decimate.GetOutput());
normalsGenerator.FlipNormalsOn();
normalsGenerator.Update();

In [10]:
cleanPolyDataFilter = vtk.vtkCleanPolyData();
cleanPolyDataFilter.SetInputConnection(normalsGenerator.GetOutputPort());
cleanPolyDataFilter.PieceInvariantOff();
cleanPolyDataFilter.ConvertLinesToPointsOff();
cleanPolyDataFilter.ConvertPolysToLinesOff();
cleanPolyDataFilter.ConvertStripsToPolysOff();
cleanPolyDataFilter.PointMergingOn();
cleanPolyDataFilter.Update();

In [11]:
connectivityFilter = vtk.vtkPolyDataConnectivityFilter();
connectivityFilter.SetInputConnection(cleanPolyDataFilter.GetOutputPort());
connectivityFilter.ScalarConnectivityOff();
connectivityFilter.SetExtractionModeToLargestRegion();
connectivityFilter.Update();

In [12]:
mesh = connectivityFilter.GetOutput()

In [13]:
renderWindow = vtk.vtkRenderWindow();
renderer = vtk.vtkRenderer();
interactorStyle = vtk.vtkInteractorStyleTrackballCamera();
renderWindowInteractor = vtk.vtkRenderWindowInteractor();

renderWindow.AddRenderer(renderer);
renderWindow.SetSize(512*2,316*2);
renderWindowInteractor.SetInteractorStyle(interactorStyle);
renderWindowInteractor.SetRenderWindow(renderWindow);

In [14]:
mapper = vtk.vtkPolyDataMapper();
mapper.SetInputData(mesh);
actor = vtk.vtkActor();
actor.SetMapper(mapper);
actor.GetProperty().SetColor(1.0, 0.0, 0.0);
renderer.AddActor(actor);

In [15]:
renderer.SetBackground(0,0,0);
renderer.ResetCamera();
#renderer.GetActiveCamera().SetPosition(-469.4609169617528, 120.71326606030519, 58.537364440710505)
#(-469.4609169617528, 120.71326606030519, 58.537364440710505) # for right pelvis
renderer.GetActiveCamera().SetViewAngle(30)
#30.0 # for right pelvis
renderWindow.Render();

# # screenshot capture code: (not used here though)
# w2if = vtk.vtkWindowToImageFilter()
# w2if.SetInput(renderWindow)
# w2if.Update()
 
# writer = vtk.vtkPNGWriter()
# writer.SetFileName(fileDir.split('/')[5]+'.png')
# writer.SetInputData(w2if.GetOutput())
# writer.Write()

renderWindowInteractor.Start();

For writing stl file

In [16]:
#writer = vtk.vtkSTLWriter()
#writer.SetInputData(mesh)
#writer.SetFileTypeToBinary()
#writer.SetFileName("pelvis_5mm.stl")
#writer.Write()